In [1]:


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
import tensorflow as tf
import numpy as np
import os
import cv2
from glob import glob
from collections import defaultdict

# Path to quantized TFLite model
model_path = "/content/drive/MyDrive/saved_model/best_saved_model/best_integer_quant.tflite"

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Determine input shape
input_shape = input_details[0]['shape']
img_height, img_width = input_shape[1], input_shape[2]

# Path to test dataset
test_root = "/content/drive/MyDrive/yolo_classification_dataset/test/"

# Class names based on folder structure
class_names = sorted(os.listdir(test_root))
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}

# Accuracy counters
correct = 0
total = 0
per_class = defaultdict(lambda: {"correct": 0, "total": 0})

print("🔍 Evaluating model on test set...\n")

# Loop through each class
for cls_name in class_names:
    cls_folder = os.path.join(test_root, cls_name)
    image_paths = glob(os.path.join(cls_folder, "*.jpg"))  # or .png

    for img_path in image_paths:
        # Load and preprocess image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.expand_dims(img.astype(np.float32) / 255.0, axis=0)

        # If model expects channel-first input (1, 3, H, W), transpose it
        if input_shape[1] == 3 and input_shape[3] != 3:
            img = np.transpose(img, (0, 3, 1, 2))

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = int(np.argmax(output))
        actual_class = class_to_idx[cls_name]

        total += 1
        per_class[cls_name]["total"] += 1

        if predicted_class == actual_class:
            correct += 1
            per_class[cls_name]["correct"] += 1

#  Final Accuracy
overall_acc = correct / total * 100
print(f" Overall Accuracy: {correct}/{total} = {overall_acc:.2f}%\n")

#  Per-class Accuracy
print("Per-Class Accuracy:")
for cls_name in class_names:
    c = per_class[cls_name]["correct"]
    t = per_class[cls_name]["total"]
    acc = (c / t * 100) if t else 0
    print(f"  {cls_name}: {c}/{t} = {acc:.2f}%")


🔍 Evaluating model on test set...

 Overall Accuracy: 99/187 = 52.94%

Per-Class Accuracy:
  1: 0/21 = 0.00%
  10: 0/10 = 0.00%
  11: 0/13 = 0.00%
  12: 0/10 = 0.00%
  13: 0/8 = 0.00%
  14: 0/11 = 0.00%
  15: 0/13 = 0.00%
  2: 16/16 = 100.00%
  3: 23/23 = 100.00%
  4: 7/9 = 77.78%
  5: 8/8 = 100.00%
  6: 7/7 = 100.00%
  7: 13/13 = 100.00%
  8: 10/10 = 100.00%
  9: 15/15 = 100.00%


In [15]:
import tensorflow as tf
import numpy as np
import os
import cv2
from glob import glob
from collections import defaultdict

# Path to quantized TFLite model
model_path = "/content/drive/MyDrive/saved_model/best_saved_model/best_integer_quant.tflite"

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Determine input shape
input_shape = input_details[0]['shape']
img_height, img_width = input_shape[1], input_shape[2]

# Path to test dataset
test_root = "/content/drive/MyDrive/yolo_classification_dataset/train/"

# Class names based on folder structure
class_names = sorted(os.listdir(test_root))
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}

# Accuracy counters
correct = 0
total = 0
per_class = defaultdict(lambda: {"correct": 0, "total": 0})

print("🔍 Evaluating model on test set...\n")

# Loop through each class
for cls_name in class_names:
    cls_folder = os.path.join(test_root, cls_name)
    image_paths = glob(os.path.join(cls_folder, "*.jpg"))  # or .png

    for img_path in image_paths:
        # Load and preprocess image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.expand_dims(img.astype(np.float32) / 255.0, axis=0)

        # If model expects channel-first input (1, 3, H, W), transpose it
        if input_shape[1] == 3 and input_shape[3] != 3:
            img = np.transpose(img, (0, 3, 1, 2))

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = int(np.argmax(output))
        actual_class = class_to_idx[cls_name]

        total += 1
        per_class[cls_name]["total"] += 1

        if predicted_class == actual_class:
            correct += 1
            per_class[cls_name]["correct"] += 1

#  Final Accuracy
overall_acc = correct / total * 100
print(f" Overall Accuracy: {correct}/{total} = {overall_acc:.2f}%\n")

#  Per-class Accuracy
print("Per-Class Accuracy:")
for cls_name in class_names:
    c = per_class[cls_name]["correct"]
    t = per_class[cls_name]["total"]
    acc = (c / t * 100) if t else 0
    print(f"  {cls_name}: {c}/{t} = {acc:.2f}%")


🔍 Evaluating model on test set...

 Overall Accuracy: 810/826 = 98.06%

Per-Class Accuracy:
  data_10_person: 42/42 = 100.00%
  data_11_person: 59/59 = 100.00%
  data_12_person: 46/51 = 90.20%
  data_13_person: 41/41 = 100.00%
  data_14_person: 49/49 = 100.00%
  data_15_person: 51/51 = 100.00%
  data_1_person: 75/75 = 100.00%
  data_2_person: 61/64 = 95.31%
  data_3_person: 81/81 = 100.00%
  data_4_person: 51/59 = 86.44%
  data_5_person: 43/43 = 100.00%
  data_6_person: 39/39 = 100.00%
  data_7_person: 59/59 = 100.00%
  data_8_person: 52/52 = 100.00%
  data_9_person: 61/61 = 100.00%


In [16]:
import tensorflow as tf
import numpy as np
import os
import cv2
from glob import glob
from collections import defaultdict

# Path to quantized TFLite model
model_path = "/content/drive/MyDrive/saved_model/best_saved_model/best_int8.tflite"

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Determine input shape
input_shape = input_details[0]['shape']
img_height, img_width = input_shape[1], input_shape[2]

# Path to test dataset
test_root = "/content/drive/MyDrive/yolo_classification_dataset/test/"

# Class names based on folder structure
class_names = sorted(os.listdir(test_root))
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}

# Accuracy counters
correct = 0
total = 0
per_class = defaultdict(lambda: {"correct": 0, "total": 0})

print("🔍 Evaluating model on test set...\n")

# Loop through each class
for cls_name in class_names:
    cls_folder = os.path.join(test_root, cls_name)
    image_paths = glob(os.path.join(cls_folder, "*.jpg"))  # or .png

    for img_path in image_paths:
        # Load and preprocess image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.expand_dims(img.astype(np.float32) / 255.0, axis=0)

        # If model expects channel-first input (1, 3, H, W), transpose it
        if input_shape[1] == 3 and input_shape[3] != 3:
            img = np.transpose(img, (0, 3, 1, 2))

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = int(np.argmax(output))
        actual_class = class_to_idx[cls_name]

        total += 1
        per_class[cls_name]["total"] += 1

        if predicted_class == actual_class:
            correct += 1
            per_class[cls_name]["correct"] += 1

#  Final Accuracy
overall_acc = correct / total * 100
print(f" Overall Accuracy: {correct}/{total} = {overall_acc:.2f}%\n")

#  Per-class Accuracy
print("Per-Class Accuracy:")
for cls_name in class_names:
    c = per_class[cls_name]["correct"]
    t = per_class[cls_name]["total"]
    acc = (c / t * 100) if t else 0
    print(f"  {cls_name}: {c}/{t} = {acc:.2f}%")


🔍 Evaluating model on test set...

 Overall Accuracy: 101/187 = 54.01%

Per-Class Accuracy:
  1: 0/21 = 0.00%
  10: 0/10 = 0.00%
  11: 0/13 = 0.00%
  12: 0/10 = 0.00%
  13: 0/8 = 0.00%
  14: 0/11 = 0.00%
  15: 0/13 = 0.00%
  2: 16/16 = 100.00%
  3: 23/23 = 100.00%
  4: 9/9 = 100.00%
  5: 8/8 = 100.00%
  6: 7/7 = 100.00%
  7: 13/13 = 100.00%
  8: 10/10 = 100.00%
  9: 15/15 = 100.00%


In [17]:

# Path to quantized TFLite model
model_path = "/content/drive/MyDrive/saved_model/best_saved_model/best_float16.tflite"

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Determine input shape
input_shape = input_details[0]['shape']
img_height, img_width = input_shape[1], input_shape[2]

# Path to test dataset
test_root = "/content/drive/MyDrive/yolo_classification_dataset/test/"

# Class names based on folder structure
class_names = sorted(os.listdir(test_root))
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}

# Accuracy counters
correct = 0
total = 0
per_class = defaultdict(lambda: {"correct": 0, "total": 0})

print("🔍 Evaluating model on test set...\n")

# Loop through each class
for cls_name in class_names:
    cls_folder = os.path.join(test_root, cls_name)
    image_paths = glob(os.path.join(cls_folder, "*.jpg"))  # or .png

    for img_path in image_paths:
        # Load and preprocess image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.expand_dims(img.astype(np.float32) / 255.0, axis=0)

        # If model expects channel-first input (1, 3, H, W), transpose it
        if input_shape[1] == 3 and input_shape[3] != 3:
            img = np.transpose(img, (0, 3, 1, 2))

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = int(np.argmax(output))
        actual_class = class_to_idx[cls_name]

        total += 1
        per_class[cls_name]["total"] += 1

        if predicted_class == actual_class:
            correct += 1
            per_class[cls_name]["correct"] += 1

#  Final Accuracy
overall_acc = correct / total * 100
print(f" Overall Accuracy: {correct}/{total} = {overall_acc:.2f}%\n")

#  Per-class Accuracy
print("Per-Class Accuracy:")
for cls_name in class_names:
    c = per_class[cls_name]["correct"]
    t = per_class[cls_name]["total"]
    acc = (c / t * 100) if t else 0
    print(f"  {cls_name}: {c}/{t} = {acc:.2f}%")


🔍 Evaluating model on test set...

 Overall Accuracy: 101/187 = 54.01%

Per-Class Accuracy:
  1: 0/21 = 0.00%
  10: 0/10 = 0.00%
  11: 0/13 = 0.00%
  12: 0/10 = 0.00%
  13: 0/8 = 0.00%
  14: 0/11 = 0.00%
  15: 0/13 = 0.00%
  2: 16/16 = 100.00%
  3: 23/23 = 100.00%
  4: 9/9 = 100.00%
  5: 8/8 = 100.00%
  6: 7/7 = 100.00%
  7: 13/13 = 100.00%
  8: 10/10 = 100.00%
  9: 15/15 = 100.00%
